In [ ]:
# FEniCS dataset generator for 2D Advection–Diffusion pollutant transport
# PDE: phi * c_t + div(-D grad c + v c) = 0  in Omega=[0,1]^2
# Output: full sequences and stepper pairs for operator learning

from fenics import *
import numpy as np
from scipy.interpolate import griddata
from tqdm import tqdm

# --------------------------- PARAMETERS ---------------------------
grid_size   = 32          # output H=W=32
nx = ny      = grid_size  # FE mesh resolution (match output for simplicity)
T_final     = 1.0         # final time
T_steps     = 20          # number of steps → T+1 frames
dt          = T_final / T_steps
theta       = 0.5         # Crank–Nicolson weighting for advection (0.5)
origin_meta = "lower"
layout_meta = "THW"       # we store sequences as (T+1, H, W)

# dataset sizes (adjust as needed)
train_per_combo = 32
test_per_combo  = 3

# mode sets (velocity pattern, inlet profile, fields pattern)
modes_v   = ["uniform", "shear", "vortex"]
modes_in  = ["const", "pulse"]
modes_D   = ["const", "smooth"]
modes_phi = ["const", "smooth"]

# bounds
phi_minmax = (0.4, 1.0)    # porosity range
D_minmax   = (1e-4, 5e-3)  # diffusivity range
c_init_amp = (0.5, 1.0)    # initial plume amplitude

# --------------------------- FENICS SETUP ---------------------------
set_log_level(LogLevel.ERROR)

mesh = UnitSquareMesh(nx, ny)
V    = FunctionSpace(mesh, "Lagrange", 1)
VV   = VectorFunctionSpace(mesh, "Lagrange", 1)

c  = TrialFunction(V)
w  = TestFunction(V)

# Regular output grid (origin lower-left)
xv = np.linspace(0, 1, grid_size)
yv = np.linspace(0, 1, grid_size)
xx, yy = np.meshgrid(xv, yv, indexing="xy")
grid_points = np.stack([xx.ravel(), yy.ravel()], axis=-1)

dof_coords = V.tabulate_dof_coordinates()

# Boundary tags
left = CompiledSubDomain("near(x[0], 0.0) && on_boundary")

# ------------------------ RANDOM HELPERS ---------------------------
def rng_from_seed(seed):
    return np.random.default_rng(int(seed))

def smooth_rand_field_V(rng, low, high):
    """Make a smooth-ish scalar field in V by summing a few Gaussians."""
    k = rng.integers(2, 4)
    parts = []
    for _ in range(k):
        A = float(rng.uniform(low, high))
        x0, y0 = rng.random(2)
        sigma = float(rng.uniform(0.15, 0.35))
        parts.append(f"{A}*exp(-(pow(x[0]-{x0},2)+pow(x[1]-{y0},2))/(2*pow({sigma},2)))")
    expr = Expression(" + ".join(parts), degree=3)
    fn = interpolate(expr, V)
    return fn

# ------------------------ COEFFICIENTS ------------------------------
def build_phi(mode, seed):
    rng = rng_from_seed(seed)
    if mode == "const":
        val = float(rng.uniform(*phi_minmax))
        return interpolate(Constant(val), V)
    else:
        # smooth in [phi_min, phi_max]
        base = smooth_rand_field_V(rng, 0.0, 1.0)
        arr = base.vector().get_local()
        arr = (arr - arr.min()) / (arr.max() - arr.min() + 1e-12)
        arr = phi_minmax[0] + arr * (phi_minmax[1] - phi_minmax[0])
        fn = Function(V); fn.vector()[:] = arr
        return fn

def build_D(mode, seed):
    rng = rng_from_seed(seed)
    if mode == "const":
        val = float(rng.uniform(*D_minmax))
        return interpolate(Constant(val), V)
    else:
        base = smooth_rand_field_V(rng, 0.0, 1.0)
        arr = base.vector().get_local()
        arr = (arr - arr.min()) / (arr.max() - arr.min() + 1e-12)
        arr = D_minmax[0] + arr * (D_minmax[1] - D_minmax[0])
        fn = Function(V); fn.vector()[:] = arr
        return fn

def build_velocity(mode, seed):
    """
    Return (vx_expr, vy_expr) as Expressions with time parameter 't'.
    Units ~ O(1). Ensure mostly left->right to match inlet on left.
    """
    rng = rng_from_seed(seed)
    if mode == "uniform":
        Ux = float(rng.uniform(0.3, 0.7))
        Uy = float(rng.uniform(-0.1, 0.1))
        vx = Expression("Ux", degree=1, Ux=Ux, t=0.0)
        vy = Expression("Uy", degree=1, Uy=Uy, t=0.0)
    elif mode == "shear":
        Ux = float(rng.uniform(0.4, 0.8))
        vy0 = float(rng.uniform(-0.1, 0.1))
        vx = Expression("Ux*(1.0 + 0.5*sin(2*pi*x[1]))", degree=2, Ux=Ux, t=0.0, pi=np.pi)
        vy = Expression("vy0", degree=1, vy0=vy0, t=0.0)
    else:  # vortex cell with mean drift to the right
        Ux = float(rng.uniform(0.2, 0.5))
        A  = float(rng.uniform(0.2, 0.6))
        vx = Expression("Ux + A*sin(2*pi*x[1])", degree=2, Ux=Ux, A=A, t=0.0, pi=np.pi)
        vy = Expression("-A*sin(2*pi*x[0])", degree=2, A=A, t=0.0, pi=np.pi)
    return vx, vy

def update_velocity_functions(vx_expr, vy_expr, t, v_fun):
    vx_expr.t = t
    vy_expr.t = t
    v_fun.assign(project(as_vector((vx_expr, vy_expr)), VV))

def inlet_bc_expr(mode, seed):
    rng = rng_from_seed(seed)
    if mode == "const":
        cin = float(rng.uniform(0.5, 1.0))
        return Expression("cin", degree=1, cin=cin, t=0.0)
    else:  # pulse in time at inlet
        cin = float(rng.uniform(0.5, 1.0))
        omega = float(rng.uniform(2.0, 6.0)*np.pi)
        return Expression("cin * (0.5 + 0.5*sin(omega*t))", degree=2, cin=cin, omega=omega, t=0.0)

def initial_c_plume(seed):
    rng = rng_from_seed(seed)
    A  = float(rng.uniform(*c_init_amp))
    x0, y0 = rng.random(2)
    sig = float(rng.uniform(0.05, 0.15))
    expr = Expression("A*exp(-(pow(x[0]-x0,2)+pow(x[1]-y0,2))/(2*pow(sig,2)))",
                      degree=3, A=A, x0=x0, y0=y0, sig=sig)
    return interpolate(expr, V)

# ------------------------ WEAK FORM (θ-scheme) -----------------------
# Unknown c^{n+1} = c1, known c^n = c0
c0 = Function(V)  # will hold current solution
phi = Function(V)
D   = Function(V)
v   = Function(VV)  # vector field at time n+theta

# Skew-symmetric advection bilinear form helper:
def adv_biform(u, vfield, test):
    # 0.5[ (v·∇u) test - (v·∇test) u ] dx
    return 0.5*( dot(vfield, grad(u))*test - dot(vfield, grad(test))*u )*dx

# Build A(c1, w) and L(w)
def build_forms():
    c1 = c
    # LHS
    A_form = (phi/dt)*c1*w*dx + D*dot(grad(c1), grad(w))*dx \
             + theta * adv_biform(c1, v, w)
    # RHS
    L_form = (phi/dt)*c0*w*dx - (1.0 - theta) * adv_biform(c0, v, w)
    return A_form, L_form

# ------------------------ INTERPOLATION TO GRID ----------------------
def to_grid(dof_vals):
    return griddata(dof_coords, dof_vals, grid_points, method="cubic").reshape(grid_size, grid_size)

def to_grid_seq(dofs_seq):
    # dofs_seq: (T+1, num_dofs) -> (T+1, H, W)
    return np.stack([to_grid(dofs_seq[n]) for n in range(dofs_seq.shape[0])], axis=0)

# ------------------------ SIMULATION PER SAMPLE ----------------------
def simulate_one(mode_v, mode_in, mode_D, mode_phi, seed_base):
    """
    Returns:
      c_seq:  (T+1, H, W)
      v_seq:  (T+1, H, W, 2)  (vx, vy)
      D_grid: (H, W)
      phi_g:  (H, W)
    """
    # Seeds per component (deterministic)
    seed_vel = seed_base + 100
    seed_in  = seed_base + 200
    seed_D   = seed_base + 300
    seed_phi = seed_base + 400
    seed_ic  = seed_base + 500

    # Coefficients
    phi_fn = build_phi(mode_phi, seed_phi)  # in V
    D_fn   = build_D(mode_D, seed_D)        # in V
    phi.assign(phi_fn); D.assign(D_fn)

    # Velocity expressions and function
    vx_expr, vy_expr = build_velocity(mode_v, seed_vel)
    v_fun = Function(VV); update_velocity_functions(vx_expr, vy_expr, 0.0, v_fun)
    v.assign(v_fun)

    # Inlet BC
    bc_expr = inlet_bc_expr(mode_in, seed_in)
    bc = DirichletBC(V, bc_expr, left)

    # Initial condition
    c_init = initial_c_plume(seed_ic)
    c0.assign(c_init)

    # Prepare forms & solvers
    A_form, L_form = build_forms()
    A = assemble(A_form)   # initial assembly (will re-assemble as v changes)
    b = None
    bc.apply(A)

    # Storage
    num_dofs = V.dim()
    c_dofs_seq = np.zeros((T_steps+1, num_dofs))
    vx_seq = np.zeros((T_steps+1, num_dofs))
    vy_seq = np.zeros((T_steps+1, num_dofs))

    # t=0 store
    c_dofs_seq[0] = c0.vector().get_local()
    vx0, vy0 = v_fun.split(deepcopy=True)
    vx_seq[0] = project(vx0, V).vector().get_local()
    vy_seq[0] = project(vy0, V).vector().get_local()

    # Time-march
    c1 = Function(V)
    for n in range(1, T_steps+1):
        t_nhalf = (n - 0.5)*dt

        # update velocity (time param) and inlet bc(t)
        update_velocity_functions(vx_expr, vy_expr, t_nhalf, v_fun)
        v.assign(v_fun)
        bc_expr.t = t_nhalf

        # Re-assemble system with updated v
        A = assemble(A_form)
        b = assemble(L_form)
        bc.apply(A, b)

        solve(A, c1.vector(), b)

        # shift in time
        c0.assign(c1)

        # store fields
        c_dofs_seq[n] = c1.vector().get_local()
        vx_t, vy_t = v_fun.split(deepcopy=True)
        vx_seq[n] = project(vx_t, V).vector().get_local()
        vy_seq[n] = project(vy_t, V).vector().get_local()

    # Interpolate to regular grid
    c_seq  = to_grid_seq(c_dofs_seq)                     # (T+1,H,W)
    D_grid = to_grid(D_fn.vector().get_local())          # (H,W)
    phi_g  = to_grid(phi_fn.vector().get_local())        # (H,W)

    # Velocity on grid (framewise)
    vx_grid = to_grid_seq(vx_seq)                        # (T+1,H,W)
    vy_grid = to_grid_seq(vy_seq)
    v_seq   = np.stack([vx_grid, vy_grid], axis=-1)      # (T+1,H,W,2)

    return c_seq, v_seq, D_grid, phi_g

# ------------------------ STEPPER PAIRS ------------------------------
def build_stepper_pairs(c_seq, v_seq, D_grid, phi_g):
    """
    Inputs:
      c_seq:  (T+1,H,W)
      v_seq:  (T+1,H,W,2)
      D_grid: (H,W)
      phi_g:  (H,W)
    Returns:
      X: (N_t, 5, H, W) where channels=[c^n, vx^n, vy^n, D, phi]
      Y: (N_t, H, W)   with Y = c^{n+1}
      where n=0..T-1  (N_t = T)
    """
    T1, H, W = c_seq.shape
    T = T1 - 1
    X_list, Y_list = [], []
    for n in range(T):
        Xn = np.stack([ c_seq[n], v_seq[n,:,:,0], v_seq[n,:,:,1], D_grid, phi_g ], axis=0)
        Yn = c_seq[n+1]
        X_list.append(Xn); Y_list.append(Yn)
    X = np.stack(X_list, axis=0)  # (T,5,H,W)
    Y = np.stack(Y_list, axis=0)  # (T,H,W)
    return X, Y

# ------------------------ DATASET BUILDERS ---------------------------
def generate_split(per_combo, tag="train"):
    """
    Loop all mode combos deterministically and generate per_combo samples each.
    Returns stacked arrays and also saves to disk:
      {tag}_advec_seq.npz  (full sequences)
      {tag}_advec_pairs.npz (stepper pairs)
    """
    seq_samples = []
    pair_X, pair_Y = [], []

    print(f"Generating {tag} set...")
    count = 0
    for mv in modes_v:
        for mi in modes_in:
            for mD in modes_D:
                for mphi in modes_phi:
                    for k in tqdm(range(per_combo), desc=f"{mv},{mi},{mD},{mphi}"):
                        seed_base = 10_000 + 10_000*count + k
                        c_seq, v_seq, D_grid, phi_g = simulate_one(mv, mi, mD, mphi, seed_base)
                        seq_samples.append( (c_seq, v_seq, D_grid, phi_g) )
                        X, Y = build_stepper_pairs(c_seq, v_seq, D_grid, phi_g)
                        pair_X.append(X); pair_Y.append(Y)
                    count += 1

    # Stack
    c_all   = np.stack([s[0] for s in seq_samples])        # (B,T+1,H,W)
    v_all   = np.stack([s[1] for s in seq_samples])        # (B,T+1,H,W,2)
    D_all   = np.stack([s[2] for s in seq_samples])        # (B,H,W)
    phi_all = np.stack([s[3] for s in seq_samples])        # (B,H,W)

    X_all = np.concatenate(pair_X, axis=0)                 # (N,5,H,W)
    Y_all = np.concatenate(pair_Y, axis=0)                 # (N,H,W)

    # Save sequences
    np.savez_compressed(
        f"{tag}_advec_seq.npz",
        c=c_all, v=v_all, D=D_all, phi=phi_all,
        layout=layout_meta, origin=origin_meta,
        T_plus_1=T_steps+1, dt=dt, H=grid_size, W=grid_size
    )
    # Save stepper pairs
    np.savez_compressed(
        f"{tag}_advec_pairs.npz",
        X=X_all, Y=Y_all,
        channels="c,vx,vy,D,phi", layout="CHW", origin=origin_meta,
        dt=dt, H=grid_size, W=grid_size
    )

    print(f"Saved '{tag}_advec_seq.npz' and '{tag}_advec_pairs.npz'")
    print(f"{tag} (seq): c {c_all.shape}, v {v_all.shape}, D {D_all.shape}, phi {phi_all.shape}")
    print(f"{tag} (pairs): X {X_all.shape}, Y {Y_all.shape}")

# ------------------------ MAIN --------------------------------------
if __name__ == "__main__":
    # Deterministic TEST set
    generate_split(test_per_combo, tag="test")

    # TRAIN set (larger)
    generate_split(train_per_combo, tag="train")

Generating test set...


uniform,const,const,const:   0%|          | 0/3 [00:00<?, ?it/s]

Calling FFC just-in-time (JIT) compiler, this may take some time.
Calling FFC just-in-time (JIT) compiler, this may take some time.


vortex,pulse,smooth,smooth: 100%|██████████| 3/3 [00:04<00:00,  1.46s/it]


Saved 'test_advec_seq.npz' and 'test_advec_pairs.npz'
test (seq): c (72, 21, 32, 32), v (72, 21, 32, 32, 2), D (72, 32, 32), phi (72, 32, 32)
test (pairs): X (1440, 5, 32, 32), Y (1440, 32, 32)
Generating train set...


vortex,pulse,smooth,smooth: 100%|██████████| 32/32 [02:15<00:00,  4.23s/it]


Saved 'train_advec_seq.npz' and 'train_advec_pairs.npz'
train (seq): c (768, 21, 32, 32), v (768, 21, 32, 32, 2), D (768, 32, 32), phi (768, 32, 32)
train (pairs): X (15360, 5, 32, 32), Y (15360, 32, 32)
